In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load datasets
behaviors = pd.read_parquet("/kaggle/input/ebnerd-large/behaviors.parquet")
articles = pd.read_parquet("/kaggle/input/ebnerd-large/articles.parquet")
history = pd.read_parqyet(("/kaggle/input/ebnerd-large/history.parquet"))
print(behaviors)

          impression_id  article_id     impression_time  read_time  \
0                 47727         NaN 2023-05-21 21:35:07       20.0   
1                 47731         NaN 2023-05-21 21:32:33       13.0   
2                 47736         NaN 2023-05-21 21:33:32       17.0   
3                 47737         NaN 2023-05-21 21:38:17       27.0   
4                 47740         NaN 2023-05-21 21:36:02       48.0   
...                 ...         ...                 ...        ...   
12063885      580100982         NaN 2023-05-18 10:27:05        9.0   
12063886      580100984         NaN 2023-05-18 10:30:39       13.0   
12063887      580100987         NaN 2023-05-18 10:27:28       13.0   
12063888      580100996         NaN 2023-05-18 10:08:18       35.0   
12063889      580101002         NaN 2023-05-18 10:29:10       28.0   

          scroll_percentage  device_type  \
0                       NaN            1   
1                       NaN            1   
2                       NaN

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [3]:
behaviors_articles = pd.merge(behaviors, articles, on='article_id', how='left')
behaviors_articles = behaviors_articles[behaviors_articles['gender'].notna()]
print(behaviors_articles)

          impression_id  article_id     impression_time  read_time  \
22986           1264789   9777026.0 2023-05-23 12:46:54       14.0   
22987           1264796   9777475.0 2023-05-23 12:41:31       97.0   
22988           1264797   9775908.0 2023-05-23 12:43:09      197.0   
22989           1264798   9777026.0 2023-05-23 12:48:35        7.0   
22990           1264800   9777704.0 2023-05-23 12:37:26      126.0   
...                 ...         ...                 ...        ...   
12031770      578904408         NaN 2023-05-22 03:00:56       20.0   
12031771      578904410         NaN 2023-05-22 03:04:50       39.0   
12031772      578904413         NaN 2023-05-22 03:03:55       29.0   
12031774      578904445   9682487.0 2023-05-22 03:25:39       62.0   
12031775      578904447   9766752.0 2023-05-22 03:27:25       93.0   

          scroll_percentage  device_type  \
22986                 100.0            1   
22987                 100.0            1   
22988                   NaN

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [6]:
# Proportion of male and female users
gender_proportion = behaviors_articles['gender'].value_counts(normalize=True)

# Average articles read per active day
avg_articles_per_day = behaviors_articles.groupby('gender')['article_id'].count() / behaviors_articles.groupby('gender')['user_id'].nunique()

# Average active days
avg_active_days = behaviors_articles.groupby('gender')['session_id'].nunique() / behaviors_articles.groupby('gender')['user_id'].nunique()

summary_table = pd.DataFrame({
    'Proportion of User Base': gender_proportion,
    'Avg Active Days': avg_active_days,
    'Avg Articles Read per Active Day': avg_articles_per_day
})
print(summary_table)

        Proportion of User Base  Avg Active Days  \
gender                                             
0.0                    0.876176        13.269491   
1.0                    0.123793        10.710330   
2.0                    0.000032        16.000000   

        Avg Articles Read per Active Day  
gender                                    
0.0                             8.362542  
1.0                             7.177533  
2.0                             0.000000  


In [5]:
# Explode the subcategory column
behaviors_articles_exploded = behaviors_articles.explode('subcategory')

# Top categories by gender
top_categories = behaviors_articles.groupby(['gender', 'category_str'])['article_id'].count().reset_index(name='count')
top_categories = top_categories.sort_values(by=['gender', 'count'], ascending=[True, False])

# Top subcategories by gender
# top_subcategories = behaviors_articles_exploded.groupby(['gender', 'subcategory'])['article_id'].count().reset_index(name='count')
# top_subcategories = top_subcategories.sort_values(by=['gender', 'count'], ascending=[True, False])
# top_subcategories = behaviors_articles_exploded.groupby(['gender', 'subcategory', 'category_str'])['article_id'].count().reset_index(name='count')
# top_subcategories = top_subcategories.sort_values(by=['gender', 'count'], ascending=[True, False])

# Display top 10 categories and subcategories for each gender
print("Top Categories by Gender:")
print(top_categories.groupby('gender').head(10))

# print("\nTop Subcategories by Gender:")
# print(top_subcategories.groupby('gender').head(10))

Top Categories by Gender:
    gender   category_str  count
9      0.0        nyheder  91064
16     0.0          sport  44080
17     0.0  underholdning  36644
6      0.0          krimi  32855
11     0.0          penge  13799
8      0.0       nationen   9354
7      0.0          musik   4517
3      0.0        forbrug   4310
14     0.0  sex_og_samliv   1652
15     0.0          side9    391
28     1.0        nyheder  13836
34     1.0  underholdning   7155
25     1.0          krimi   6188
33     1.0          sport   4791
30     1.0          penge   1772
27     1.0       nationen   1232
22     1.0        forbrug    709
26     1.0          musik    668
31     1.0  sex_og_samliv    141
32     1.0          side9     19

Top Subcategories by Gender:
     gender  subcategory   category_str  count
7       0.0          133        nyheder  47558
39      0.0          327          sport  19938
11      0.0          196          sport  16560
5       0.0          130        nyheder  16008
3       0.0     